# Deep Convolutional Generative Adversarial Network

## Import dependencies

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'1.3.0'

In [3]:
np.__version__

'1.13.3'

## Load in data

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [6]:
print('Training:   {:,}'.format(data.train.num_examples))
print('Testing:    {:,}'.format(data.test.num_examples))
print('Validation:  {:,}'.format(data.validation.num_examples))

Training:   55,000
Testing:    10,000
Validation:  5,000


## Define hyperparameters

In [12]:
# Inputs
image_size = 28
image_channel = 1
image_shape = (image_size, image_size, image_channel)
image_size_flat = image_size * image_size * image_channel
num_classes = 10

# Network
filter_size = 5
hidden1_filter = 32
hidden2_filter = 64
fc1_size = 1024
fc2_size = 1
dropout = 0.8


## Build the Network

### Helper functions

#### `weights` and `biases`

In [7]:
def weight(shape):
    initial = tf.truncated_normal(shape=shape, mean=0, stddev=0.5)
    return tf.Variable(initial, name='weight')

def bias(shape):
    initial = tf.zeros(shape=[shape])
    return tf.Variable(initial, name='bias')

#### `convolution` and `pooling` 

In [8]:
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(X):
    return tf.nn.max_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#### `flatten` layer

In [9]:
def flatten(layer):
    layer_shape = layer.get_shape()
    num_features = np.array(layer_shape[1:4], dtype=int).prod()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

#### Layers

In [10]:
# convolutional layer
def conv_layer(prev_layer, prev_filter, layer_filter, layer_name, use_pool=True):
    with tf.name_scope(layer_name):
        W = weight(shape=[filter_size, filter_size, prev_filter, layer_filter])
        b = bias(shape=layer_filter)
        layer = tf.nn.relu(conv2d(prev_layer, W) + b)
        if use_pool:
            layer = max_pool(layer)
        return layer

# fully connected layer
def fc_layer(prev_layer, prev_size, layer_size, layer_name, use_relu=True, dropout=False):
    with tf.name_scope(layer_name):
        W = weight(shape=[prev_size, layer_size])
        b = bias(shape=layer_size)
        layer = tf.matmul(prev_layer, W) + b
        if use_relu:
            layer = tf.nn.relu(layer)
        if dropout:
            layer = tf.nn.dropout(layer, keep_prob)
        return layer

# Output layer
def output_layer(fc_layer, fc_size, num_classes):
    with tf.name_scope('output_layer'):
        W = weight(shape=[fc_size, num_classes])
        b = bias(shape=num_classes)
        logits = tf.matmul(fc_layer, W) + b
        y_pred = tf.nn.softmax(logits)
        y_pred_true = tf.argmax(y_pred, axis=1)
        return logits, y_pred_true

#### Discrimitor

In [ ]:
def discriminator(X_image, reuse=False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
    
    # Input layer => Hidden layer 1
    hidden1 = conv_layer(X_image, image_channel, hidden1_filter, 'dis_hidden_1', use_pool=True)
    # Hidden layer 1 => Hidden layer 2
    hidden2 = conv_layer(hidden1, hidden1_filter, hidden2_filter, 'dis_hidden_2', use_pool=True)
    # Flatten
    hidden2_flat, hidden2_flat_filter = flatten(hidden2)
    # Fully connected layer 1
    fc1_layer = fc_layer(hidden2_flat, hidden2_flat_filter, fc1_size, 'dis_fc_1', use_relu=True, dropout=True)
    fc2_layer = fc_layer(fc1_layer, fc1_size, fc2_size, 'dis_fc_2', use_relu=True, dropout=True)
   
    # Flattening for the fully connected layer
    